# Multi-Step Tool Use

<a target="_blank" href="https://colab.research.google.com/github/cohere-ai/notebooks/blob/main/notebooks/llmu/multi_step_tool_use.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

*Read the accompanying [article here](https://cohere.ai/blog/multi-step-tool-use/).*

Multi-step tool use happens when more than one tool is required, and the output of one tool is needed as the input to another tool.

In other words, tool-calling needs to happen in a sequence.

This is especially true in complex workflows where tasks have to be broken down into a sequence of steps.

In this notebook, we’ll build a calendar assistant that can check a user’s calendar and create new appointments based on available times. This use case requires tool calls to happen in sequence, i.e., multi-step.

It is also an example of using tools for performing not just a read operation (retrieving information) but also a write operation (taking actions).


First, let’s install the Cohere Python SDK and set up the Cohere client.

In [ ]:
! pip install cohere -q

In [18]:
import cohere
import json
co = cohere.Client("COHERE_API_KEY") # Get your API key: https://dashboard.cohere.com/api-keys

# Create Tools

Let’s now create two tools:
- A function called `list_calendar_events` to list the existing calendar events based on a given date. For simplicity, we are not going to connect to an actual calendar. So we’ll include a mock events record, which is a simple list of events in a day.
- A function called `create_calendar_event` to create a new calendar event based on the provided date, time, and duration. Again, to keep things simple, we are not going to make actual changes to a database, but instead will just return a simple success message for illustration purposes.

In [19]:
# Define the tools

def list_calendar_events(date: str):
  events = [{"start": "8:00", "end": "8:59"}, {"start": "9:00", "end": "9:59"}, {"start": "11:00", "end": "11:59"},{"start": "12:00", "end": "12:59"}]

  return {
        "existing_events": events
    }

def create_calendar_event(date: str, time: str, duration: int):
  
  return {
        "is_success": True,
        "message": f"Created a {duration} hour long event at {time} on {date}"
    }

functions_map = {
    "list_calendar_events": list_calendar_events,
    "create_calendar_event": create_calendar_event
}


Next, we define the tool schema for the two tools.

In [20]:
tools = [
    {
      "name": "list_calendar_events",
      "description": "Returns a list of existing events for the specified date, including the start time and end time for each event.",
      "parameter_definitions": {
        "date": {
          "description": "the date to list events for, formatted as mm/dd/yy",
          "type": "str",
          "required": True
        }
      }
    }, 
    {
      "name": "create_calendar_event",
      "description": "Creates a new calendar event of the specified duration at the specified time and date. A new event cannot be created on the same time as an existing event.",
      "parameter_definitions": {
        "date": {
          "description": "the date on which the event starts, formatted as mm/dd/yy",
          "type": "str",
          "required": True
        },
        "time": {
          "description": "the time of the event, formatted using 24h military time formatting",
          "type": "str",
          "required": True
        },
        "duration": {
          "description": "the number of hours the event lasts for",
          "type": "float",
          "required": True
        }
      }
    }
]

Let’s also create a custom preamble. The important part here is mentioning the date since the `list_calendar_events` tool is a simple enough version that it doesn’t do any date-handling. What we’re doing is nudging the model to use this date for the create_calendar_event tool call, which requires a date field.

In [21]:
preamble="""## Task & Context
You are a calendar assistant who helps people schedule events on their calendar. You must make sure that a new event does not overlap with any existing event.
Today is Thursday, May 23, 2024
"""

Let's create a `run_assistant` function that does the following:
- Get the user message (Step 1)
- Call the Chat endpoint for tool call generation (Step 2)
- If the response contains at least one tool call, execute the tool call(s) and get the tool results (Step 3)
- Repeat Steps 2 and 3 until there are no more tool calls
- Generate the final response with citations (Step 4)

Here, we remove the `force_single_step` argument as it is `False` by default. This indicates to the Cohere API to enable a multi-step workflow and use the right ReAct-style prompt for running the workflow.

In [34]:
model = "command-r-plus"

def run_assistant(message, chat_history=[]):
    # Step 1: Get user message
    print(f"Question:\n{message}")
    print("="*50)

    # Step 2: Generate tool calls (if any)    
    response = co.chat(
        message=message,
        model=model,
        preamble=preamble,
        tools=tools,
        chat_history=chat_history
    )

    while response.tool_calls:
        tool_calls = response.tool_calls
        
        if response.text:
            print("Intermediate response:")
            print(response.text,"\n")
        print("Tool calls:")
        for call in tool_calls:
            print(f"Tool name: {call.name} | Parameters: {call.parameters}")
        print("="*50)
        
        # Step 3: Get tool results
        tool_results = []
        for tc in tool_calls:
            tool_call = {"name": tc.name, "parameters": tc.parameters}
            tool_output = functions_map[tc.name](**tc.parameters)
            tool_results.append({"call": tool_call, "outputs": [tool_output]})
        
        # Step 4: Generate response and citations                
        response = co.chat(
            message="",
            model=model,
            preamble=preamble,
            tools=tools,
            tool_results=tool_results,
            chat_history=response.chat_history
        )

        # Append the current chat turn to the chat history
        chat_history = response.chat_history
        
    # Print final response
    print("Final response:")
    print(response.text)
    print("="*50)
    
    # Print citations (if any)
    if response.citations:
        print("Citations:")
        for citation in response.citations:
            print(citation)
        print("\nCited Documents:")
        for document in response.documents:
            print(document)
        print("="*50)
    
    return chat_history

## Simple question that doesn't require multi-step

Let’s now ask the assistant the first question, starting with a simple one about the number of meetings for the day. This requires just a single step of tool calling to check existing calendar events.

That’s exactly what the assistant does, and it gives the correct answer.

In [35]:
chat_history = run_assistant("How many meetings do I have today")

Question:
How many meetings do I have today
Intermediate response:
I will list the events for today and count how many meetings there are. 

Tool calls:
Tool name: list_calendar_events | Parameters: {'date': '05/23/2024'}
Final response:
You have four meetings today.
Citations:
start=9 end=22 text='four meetings' document_ids=['list_calendar_events:0:2:0']

Cited Documents:
{'existing_events': '[{"end":"8:59","start":"8:00"},{"end":"9:59","start":"9:00"},{"end":"11:59","start":"11:00"},{"end":"12:59","start":"12:00"}]', 'id': 'list_calendar_events:0:2:0', 'tool_name': 'list_calendar_events'}


## Multi-step

Let’s now try to ask a question that requires multi-step tool calling — this one asking the assistant to book an appointment. To complete this task, the assistant will have to first query the existing events and then use the information to create a new event that wouldn’t cause a conflict.

In the first step, the assistant calls the `list_calendar_events` tool to get a list of existing events.

In the second step, it calls the `create_calendar_event` tool to create a new event. It creates a new event at 10 a.m., which is indeed the first available slot after 9 a.m.

In [36]:
chat_history = run_assistant("Create an hour-long appointment for the first available free slot after 9am")

Question:
Create an hour-long appointment for the first available free slot after 9am
Intermediate response:
I will first check the user's calendar for 23/05/2024 to see if there are any free slots after 9am. Then, I will create an hour-long appointment for the first available free slot. 

Tool calls:
Tool name: list_calendar_events | Parameters: {'date': '05/23/2024'}
Intermediate response:
The user's calendar shows that there is a free slot after 9am, from 10:00 to 10:59. I will now create an hour-long appointment for this time slot. 

Tool calls:
Tool name: create_calendar_event | Parameters: {'date': '05/23/2024', 'duration': 1, 'time': '10:00'}
Final response:
I have created an hour-long appointment for 10:00 on 23/05/2024.
Citations:
start=18 end=39 text='hour-long appointment' document_ids=['create_calendar_event:0:4:0']
start=44 end=63 text='10:00 on 23/05/2024' document_ids=['create_calendar_event:0:4:0']

Cited Documents:
{'id': 'create_calendar_event:0:4:0', 'is_success': 't

And here’s a look at the chat history. Compared to the single-step scenario, it contains multiple pairs of CHATBOT - TOOL messages, with each pair being one step in the sequence of tool calls. In this particular case, we have two of them.

In [14]:
# Print chat history
for turn in chat_history:
    print(turn,"\n")

message='Create an hour-long appointment for the first available free slot after 9am' tool_calls=None role='USER' 

message="I will first check the user's calendar for 23/05/2024 to see when they are free after 9am. I will then create an hour-long appointment for the first available slot." tool_calls=[ToolCall(name='list_calendar_events', parameters={'date': '05/23/2024'})] role='CHATBOT' 

tool_results=[ToolResult(call=ToolCall(name='list_calendar_events', parameters={'date': '05/23/2024'}), outputs=[{'existing_events': [{'end': '8:59', 'start': '8:00'}, {'end': '9:59', 'start': '9:00'}, {'end': '11:59', 'start': '11:00'}, {'end': '12:59', 'start': '12:00'}]}])] role='TOOL' 

message='The user has an appointment from 9am to 9:59am. I will now create an hour-long appointment for the first available slot after this.' tool_calls=[ToolCall(name='create_calendar_event', parameters={'date': '05/23/2024', 'duration': 1, 'time': '10:00'})] role='CHATBOT' 

tool_results=[ToolResult(call=ToolCa

The chat history also highlights another difference between the single-step and multi-step scenarios.

Notice that in the multi-step example above, the CHATBOT turns generate a textual response at each tool-calling step (look for the message) before making the actual tool calls.

Let’s call this the model’s “intermediate response” to differentiate it from the model’s final response, which is the response that the user sees. These intermediate responses are the model’s internal reasoning logic, which guides its next course of action.

## Multi-step parallel

Let’s now look at an example of multi-step, parallel tool use. Let’s ask the assistant to create two separate events on the same day.

And here’s the assistant’s response. It follows the same two steps as in the earlier example: first, checking the existing events, and second, creating the new events.

The difference this time is that since it needs to create two events, it calls `create_calendar_event` twice within the same step. Here the assistant makes the correct judgment and tool calls are independent of each other, so they can be done in parallel.

In [15]:
chat_history = run_assistant("Create two hour-long appointments for any available time between 8am to 6pm")

Question:
Create two hour-long appointments for any available time between 8am to 6pm
Intermediate response:
I will first list the events for 23/05/2024. Then, I will create two one-hour appointments for any available time between 8am and 6pm. 

Tool calls:
Tool name: list_calendar_events | Parameters: {'date': '05/23/2024'}
Intermediate response:
The following times are available: 10:00-10:59, 13:00-17:59. I will now create two one-hour appointments for two of these time slots. 

Tool calls:
Tool name: create_calendar_event | Parameters: {'date': '05/23/2024', 'duration': 1, 'time': '10:00'}
Tool name: create_calendar_event | Parameters: {'date': '05/23/2024', 'duration': 1, 'time': '13:00'}
Final response:
I have created two one-hour appointments for 10:00 and 13:00 today.
Citations:
start=45 end=50 text='10:00' document_ids=['create_calendar_event:0:4:0']
start=55 end=60 text='13:00' document_ids=['create_calendar_event:1:4:0']

Cited Documents:
{'id': 'create_calendar_event:0:4:0',

## State management (memory)

The chat history for each turn consists of the following messages, and will accumulate with every new turn.

- The USER message
- Followed by the CHATBOT message with the list of tool calls
- Followed the TOOL message with the list of tool results
- Finally, followed by the CHATBOT message with the final response to the user


In [16]:
chat_history = run_assistant("Considering the new appointments you made, when is my next available time?", chat_history)

Question:
Considering the new appointments you made, when is my next available time?
Intermediate response:
I will list the events for 23/05/2024 and then inform the user of the next available time slot. 

Tool calls:
Tool name: list_calendar_events | Parameters: {'date': '05/23/2024'}
Final response:
Your next available time slot is 14:00.
Citations:
start=33 end=38 text='14:00' document_ids=['list_calendar_events:0:8:0']

Cited Documents:
{'existing_events': '[{"end":"8:59","start":"8:00"},{"end":"9:59","start":"9:00"},{"end":"11:59","start":"11:00"},{"end":"12:59","start":"12:00"}]', 'id': 'list_calendar_events:0:8:0', 'tool_name': 'list_calendar_events'}


In [17]:
# Print chat history
for turn in chat_history:
    print(turn,"\n")

message='Create two hour-long appointments for any available time between 8am to 6pm' tool_calls=None role='USER' 

message='I will first list the events for 23/05/2024. Then, I will create two one-hour appointments for any available time between 8am and 6pm.' tool_calls=[ToolCall(name='list_calendar_events', parameters={'date': '05/23/2024'})] role='CHATBOT' 

tool_results=[ToolResult(call=ToolCall(name='list_calendar_events', parameters={'date': '05/23/2024'}), outputs=[{'existing_events': [{'end': '8:59', 'start': '8:00'}, {'end': '9:59', 'start': '9:00'}, {'end': '11:59', 'start': '11:00'}, {'end': '12:59', 'start': '12:00'}]}])] role='TOOL' 

message='The following times are available: 10:00-10:59, 13:00-17:59. I will now create two one-hour appointments for two of these time slots.' tool_calls=[ToolCall(name='create_calendar_event', parameters={'date': '05/23/2024', 'duration': 1, 'time': '10:00'}), ToolCall(name='create_calendar_event', parameters={'date': '05/23/2024', 'duratio